# Lab 05 Logistic Classification (diabetes) - Session
* Logistic Classfication을 diabetes data를 활용하여 모델을 만들어 보도록 하겠습니다
### 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

tf.set_random_seed(777)  # for reproducibility

print(tf.__version__)

1.12.0


### 강의에 설명할 Data입니다

In [2]:
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[:, 0:-1]
y_train = xy[:, [-1]]

print(x_train.shape, y_train.shape)
print(xy)

(759, 8) (759, 1)
[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]


##  Tensorflow Session
### 위 Data를 기준으로 가설의 검증을 통해 Logistic Classification 모델을 만들도록 하겠습니다
* Tensorflow data API를 통해 학습시킬 값들을 담는다 (Batch Size는 한번에 학습시킬 Size로 정한다)

In [3]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train)).repeat()
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

### 위 Data를 기준으로 가설의 검증을 통해 Logistic Classification 모델을 만들도록 하겠습니다
* W와 b은 학습을 통해 생성되는 모델에 쓰이는 Wegith와 Bias (초기값을 variable : 0이나 Random값으로 가능 tf.random_normal([2, 1]) )

In [4]:
W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

### Sigmoid 함수를 가설로 선언합니다
* Sigmoid는 아래 그래프와 같이 0과 1의 값만을 리턴합니다 tf.sigmoid(tf.matmul(X, W) + b)와 같습니다

$$
\begin{align}
sigmoid(x) & = \frac{1}{1+e^{-x}}  \\\\\
\end{align}
$$

![sigmoid](https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg)

In [5]:
hypothesis  = tf.div(1., 1. + tf.exp(tf.matmul(features, W) + b))

### 가설을 검증할 Cost 함수를 정의합니다
$$
\begin{align}
cost(h(x),y) & = −log(h(x))  &  if  &  y=1 \\\\\
cost(h(x),y) & = -log(1−h(x))  &  if  &  y=0
\end{align}
$$

* 위 두수식을 합치면 아래과 같습니다
$$
\begin{align}
cost(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [6]:
cost = -tf.reduce_mean(labels * tf.log(hypothesis) + (1 - labels) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

### 추론한 값은 0.5를 기준(Sigmoid 그래프 참조)로 0과 1의 값을 리턴합니다.
* Sigmoid 함수를 통해 예측값이 0.5보다 크면 1을 반환하고 0.5보다 작으면 0으로 반환합니다.

In [7]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)

### 가설을 통해 실재 값과 비교한 정확도를 측정합니다

In [8]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))

### Tensorflow를 통한 실행을 위해 Session를 선언합니다.
* 위의 Data를 Cost함수를 통해 학습시킨 후 모델을 생성합니다. 

In [9]:
EPOCHS = 10001

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(EPOCHS):
        sess.run(iter.initializer)
        _, loss_value, acc_value = sess.run([train, cost, accuracy])
        if step % 1000 == 0:
            print("Iter: {}, Loss: {:.4f}, Acc: {}".format(step, loss_value, acc_value))

Iter: 0, Loss: 0.7398, Acc: 0.6495388746261597
Iter: 1000, Loss: 0.5888, Acc: 0.6837944388389587
Iter: 2000, Loss: 0.5506, Acc: 0.7048748135566711
Iter: 3000, Loss: 0.5272, Acc: 0.7351778745651245
Iter: 4000, Loss: 0.5122, Acc: 0.7536231875419617
Iter: 5000, Loss: 0.5021, Acc: 0.7602108120918274
Iter: 6000, Loss: 0.4951, Acc: 0.7654808759689331
Iter: 7000, Loss: 0.4901, Acc: 0.7707509994506836
Iter: 8000, Loss: 0.4864, Acc: 0.7747035622596741
Iter: 9000, Loss: 0.4837, Acc: 0.7694334387779236
Iter: 10000, Loss: 0.4816, Acc: 0.7667984366416931
